<a href="https://colab.research.google.com/github/prady1900/Kaggle_Competition/blob/main/kaggle_xGB_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle kernels list — user prady1900 — sort-by dateRun

!kaggle competitions download -c tabular-playground-series-feb-2021

!unzip -q train.csv.zip -d .
!unzip -q test.csv.zip -d .
!ls

Saving kaggle.json to kaggle.json
kaggle.json
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: — user prady1900 — sort-by dateRun
 82% 33.0M/40.2M [00:01<00:00, 9.26MB/s]
100% 40.2M/40.2M [00:02<00:00, 20.9MB/s]
 36% 9.00M/25.1M [00:01<00:02, 7.12MB/s]
100% 25.1M/25.1M [00:01<00:00, 14.8MB/s]
  0% 0.00/470k [00:00<?, ?B/s]
100% 470k/470k [00:00<00:00, 63.9MB/s]
kaggle.json  sample_submission.csv.zip	test.csv.zip  train.csv.zip
sample_data  test.csv			train.csv


In [2]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

In [3]:
df =pd.read_csv('train.csv')
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,I,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,F,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,N,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,K,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,F,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
cat_cols = df.iloc[:,1:11]
cat_cols[:5]

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9
0,A,B,A,A,B,D,A,E,C,I
1,B,A,A,A,B,B,A,E,A,F
2,A,A,A,C,B,D,A,B,C,N
3,A,A,A,C,B,D,A,E,G,K
4,A,B,A,A,B,B,A,E,C,F


In [6]:
lbl = LabelEncoder()
for cols in cat_cols.columns:
  lbl.fit(df[cols])
  df[cols] = lbl.transform(df[cols])
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,0,1,0,0,1,3,0,4,2,8,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,1,0,0,0,1,1,0,4,0,5,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,0,0,0,2,1,3,0,1,2,13,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,0,0,0,2,1,3,0,4,6,10,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,0,1,0,0,1,1,0,4,2,5,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
for cols in cat_cols:
  df[cols] = scaler.fit_transform(df[[cols]])
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,-0.256572,1.088415,-0.291189,-1.342193,-0.020835,1.075789,-0.141961,0.242093,-0.542108,0.247004,0.923191,0.684968,0.124454,0.217886,0.281421,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,3.897542,-0.918768,-0.291189,-1.342193,-0.020835,-0.929952,-0.141961,0.242093,-1.673138,-0.726522,0.437627,0.014213,0.357438,0.846127,0.282354,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,-0.256572,-0.918768,-0.291189,0.662896,-0.020835,1.075789,-0.141961,-5.573728,-0.542108,1.869547,0.732209,0.760122,0.454644,0.812990,0.293756,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,-0.256572,-0.918768,-0.291189,0.662896,-0.020835,1.075789,-0.141961,0.242093,1.719952,0.896021,0.705142,0.771678,0.153735,0.732893,0.769785,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,-0.256572,1.088415,-0.291189,-1.342193,-0.020835,-0.929952,-0.141961,0.242093,-0.542108,-0.726522,0.486063,0.639349,0.496212,0.354186,0.279105,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [11]:
from sklearn.decomposition import PCA

In [12]:
pca=PCA(n_components=2)

In [13]:
pca.fit(df.iloc[:,:-1])

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [14]:
x_pca=pca.transform(df.iloc[:,:-1])

In [11]:
y=df.iloc[:,-1]
y.head()

0    6.994023
1    8.071256
2    5.760456
3    7.806457
4    6.868974
Name: target, dtype: float64

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

In [18]:
x_pca.shape

(300000, 2)

In [20]:
x_pca

array([[ 2.50017577e+05, -3.18264841e-01],
       [ 2.50016577e+05, -2.35638528e+00],
       [ 2.50015577e+05, -9.81024621e-02],
       ...,
       [-2.49978423e+05,  1.65914393e+00],
       [-2.49979423e+05,  1.83856835e+00],
       [-2.49980423e+05, -1.20630502e-01]])

In [9]:
import xgboost as xgb

In [10]:
xgbr = xgb.XGBRegressor(verbosity=0) 

In [14]:

x_train = x_train.iloc[:,1:]


In [15]:
xgbr.fit(x_train, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=0)

In [16]:
score = xgbr.score(x_train, y)  
print("Training score: ", score)

Training score:  0.07902366426187213


In [17]:
df2 =  pd.read_csv('test.csv')

In [18]:
cat_cols1 = df2.iloc[:,1:11]
for cols in cat_cols1.columns:
  lbl.fit(df2[cols])
  df2[cols] = lbl.transform(df2[cols])
df2.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,0,1,0,2,1,3,0,4,4,6,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,0,1,0,2,1,3,0,4,2,11,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,0,1,0,2,1,3,0,4,2,5,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,0,0,1,0,1,3,0,4,4,5,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,0,1,0,0,1,1,0,4,4,8,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [19]:
for cols in cat_cols1:
  df2[cols] = scaler.transform(df2[[cols]])
df2.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,0,-2.349065,-2.024556,-2.349065,-1.700048,-2.024556,-1.375539,-2.349065,-1.051031,-1.051031,-0.402013,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
1,5,-2.349065,-2.024556,-2.349065,-1.700048,-2.024556,-1.375539,-2.349065,-1.051031,-1.700048,1.220530,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
2,15,-2.349065,-2.024556,-2.349065,-1.700048,-2.024556,-1.375539,-2.349065,-1.051031,-1.700048,-0.726522,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
3,16,-2.349065,-2.349065,-2.024556,-2.349065,-2.024556,-1.375539,-2.349065,-1.051031,-1.051031,-0.726522,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
4,17,-2.349065,-2.024556,-2.349065,-2.349065,-2.024556,-2.024556,-2.349065,-1.051031,-1.051031,0.247004,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [29]:
x_test_pca=pca.transform(df2)

In [20]:
ypred = xgbr.predict(df2.iloc[:,1:])

In [21]:
sub = pd.DataFrame()
sub["id"] = df2["id"]
sub["target"] = ypred
sub.head()

,id,target
0,0,7.265133
1,5,7.612457
2,15,7.378393
3,16,7.224644
4,17,7.226149


In [22]:
sub.to_csv('submission.csv', index=False)